In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.

In [3]:
# data_path = 'fra.txt'
# input_texts = []
# target_texts = []
# input_characters = set()
# target_characters = set()
# with open(data_path, 'r', encoding='utf-8') as f:
#     lines = f.read().split('\n')
# for line in lines[: min(num_samples, len(lines) - 1)]:
#     input_text, target_text, _ = line.split('\t')
#     target_text = '\t' + target_text + '\n'
#     input_texts.append(input_text)
#     target_texts.append(target_text)
#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)


In [4]:
data_path = '/content/오탈자.csv'
data = pd.read_csv(data_path, encoding='utf-8')
input_characters = set()
target_characters = set()
input_characters.add(' ')
target_characters.add(' ')
target_characters.add('<START>')
target_characters.add('<END>')
input_texts = data["Wrong"]
target_texts = '<START> ' + data["Right"] + ' <END>'

### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in data['Wrong']:
    for word in eng.split():
        if word not in input_characters:
            input_characters.add(word)

all_hindi_words=set()
for hin in data['Right']:
    for word in hin.split():
        if word not in target_characters:
            target_characters.add(word)

In [5]:
data.tail()

,Wrong,Right
36,방방곳곳,방방곡곡
37,성대묘사,성대모사
38,소매깃,소맷귀
39,세 살박이,세 살배기
40,쉽상,십상


In [6]:
input_characters

{' ',
 '?',
 '가는',
 '감자',
 '개거품',
 '결제했다.',
 '곳',
 '관련',
 '구렛나루',
 '궁시렁거리지',
 '그',
 '그만',
 '금새',
 '기다리면',
 '깨방정',
 '꺼야!',
 '끼여들기',
 '나리',
 '나중에',
 '날',
 '났다.',
 '낳다.',
 '내',
 '내가',
 '내노라하는',
 '내놓라하는',
 '너',
 '넌',
 '널판지',
 '담배를',
 '대고',
 '더종비즈온',
 '동맹이',
 '된대',
 '들었니?',
 '떨어라',
 '만나',
 '만나는구나~',
 '말대꾸야!',
 '몇일만',
 '모임이',
 '믾이',
 '바람에',
 '반ㄴ갑습니다.',
 '방방곳곳',
 '밪으세요.',
 '병이',
 '복',
 '뵈요.',
 '불리운다.',
 '비가',
 '사단이',
 '사장이',
 '살박이',
 '새해',
 '서류를',
 '설겆이',
 '성대묘사',
 '세',
 '소매깃',
 '쉽상',
 '안ㄴ녕하세요.',
 '알았다.',
 '양년',
 '양년치킨',
 '어따',
 '어떻해',
 '어음결재',
 '어의가',
 '언제니',
 '업습니다.',
 '없네',
 '예기',
 '오는',
 '오랫만에',
 '왠만하면',
 '이',
 '좀',
 '좋은',
 '줄',
 '창워느로',
 '천재라고',
 '출발하겠습니다.',
 '출짱',
 '파토가',
 '피다.',
 '회사의'}

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 41
Number of unique input tokens: 87
Number of unique output tokens: 88
Max sequence length for inputs: 25
Max sequence length for outputs: 39


In [9]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [10]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [11]:
input_token_index

{' ': 0,
 '?': 1,
 '가는': 2,
 '감자': 3,
 '개거품': 4,
 '결제했다.': 5,
 '곳': 6,
 '관련': 7,
 '구렛나루': 8,
 '궁시렁거리지': 9,
 '그': 10,
 '그만': 11,
 '금새': 12,
 '기다리면': 13,
 '깨방정': 14,
 '꺼야!': 15,
 '끼여들기': 16,
 '나리': 17,
 '나중에': 18,
 '날': 19,
 '났다.': 20,
 '낳다.': 21,
 '내': 22,
 '내가': 23,
 '내노라하는': 24,
 '내놓라하는': 25,
 '너': 26,
 '넌': 27,
 '널판지': 28,
 '담배를': 29,
 '대고': 30,
 '더종비즈온': 31,
 '동맹이': 32,
 '된대': 33,
 '들었니?': 34,
 '떨어라': 35,
 '만나': 36,
 '만나는구나~': 37,
 '말대꾸야!': 38,
 '몇일만': 39,
 '모임이': 40,
 '믾이': 41,
 '바람에': 42,
 '반ㄴ갑습니다.': 43,
 '방방곳곳': 44,
 '밪으세요.': 45,
 '병이': 46,
 '복': 47,
 '뵈요.': 48,
 '불리운다.': 49,
 '비가': 50,
 '사단이': 51,
 '사장이': 52,
 '살박이': 53,
 '새해': 54,
 '서류를': 55,
 '설겆이': 56,
 '성대묘사': 57,
 '세': 58,
 '소매깃': 59,
 '쉽상': 60,
 '안ㄴ녕하세요.': 61,
 '알았다.': 62,
 '양년': 63,
 '양년치킨': 64,
 '어따': 65,
 '어떻해': 66,
 '어음결재': 67,
 '어의가': 68,
 '언제니': 69,
 '업습니다.': 70,
 '없네': 71,
 '예기': 72,
 '오는': 73,
 '오랫만에': 74,
 '왠만하면': 75,
 '이': 76,
 '좀': 77,
 '좋은': 78,
 '줄': 79,
 '창워느로': 80,
 '천재라고': 81,
 '출발하겠습니다.': 82,
 '출짱': 83,
 '

In [12]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text.split()):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.
    for t, char in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=300, validation_split=0.2)
# Save model
model.save('s2s.h5')

Epoch 1/300
1/1 [==============================] - 5s 5s/step - loss: 4.4549 - accuracy: 8.0128e-04 - val_loss: 3.7601 - val_accuracy: 0.9459
Epoch 2/300
1/1 [==============================] - 0s 328ms/step - loss: 3.7934 - accuracy: 0.9127 - val_loss: 0.8029 - val_accuracy: 0.9459
Epoch 3/300
1/1 [==============================] - 0s 324ms/step - loss: 0.9196 - accuracy: 0.9127 - val_loss: 0.2714 - val_accuracy: 0.9459
Epoch 4/300
1/1 [==============================] - 0s 308ms/step - loss: 0.5244 - accuracy: 0.9127 - val_loss: 0.3280 - val_accuracy: 0.9459
Epoch 5/300
1/1 [==============================] - 0s 313ms/step - loss: 0.4498 - accuracy: 0.9127 - val_loss: 0.2296 - val_accuracy: 0.9459
Epoch 6/300
1/1 [==============================] - 0s 324ms/step - loss: 0.4292 - accuracy: 0.9127 - val_loss: 0.2679 - val_accuracy: 0.9687
Epoch 7/300
1/1 [==============================] - 0s 317ms/step - loss: 0.4066 - accuracy: 0.9239 - val_loss: 0.2360 - val_accuracy: 0.9687
Epoch 8/300


In [15]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [16]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [17]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['<START>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char + " "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<END>' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    return decoded_sentence.strip()

In [20]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 창워느로 출짱 가는 나리 언제니 ?
Decoded sentence: 내가 이 사달이 날 줄 알았다. <END>
-
Input sentence: 새해 복 믾이 밪으세요.
Decoded sentence: 개방정 좀 그만 떨어라 <END>
-
Input sentence: 어의가 없네 ?
Decoded sentence: 며칠만 기다리면 된대 <END>
-
Input sentence: 깨방정 좀 그만 떨어라
Decoded sentence: 개방정 좀 그만 떨어라 <END>
-
Input sentence: 왠만하면
Decoded sentence: 내로라하는 <END>
-
Input sentence: 어따 대고 말대꾸야!
Decoded sentence: 얻다 대고 말대꾸야! <END>
-
Input sentence: 오랫만에 만나는구나~
Decoded sentence: 나중에 봬요. <END>
-
Input sentence: 비가 오는 바람에 모임이 파토가 났다.
Decoded sentence: 개방정 좀 그만 떨어라 <END>
-
Input sentence: 나중에 뵈요.
Decoded sentence: 나중에 봬요. <END>
-
Input sentence: 내가 이 사단이 날 줄 알았다.
Decoded sentence: 내가 이 사달이 날 줄 알았다. <END>
-
Input sentence: 어떻해
Decoded sentence: 내로라하는 <END>
-
Input sentence: 넌 내 꺼야!
Decoded sentence: 며칠만 기다리면 된대 <END>
-
Input sentence: 몇일만 기다리면 된대
Decoded sentence: 며칠만 기다리면 된대 <END>
-
Input sentence: 너 그 예기 들었니?
Decoded sentence: 내가 이 사달이 날 줄 알았다. <END>
-
Input sentence: 병이 낳다.
Decoded sentence: 담배를 피우다. <END>
-
Input sentence